In [11]:
import platform

import pytest
import torch

from mmaction.registry import MODELS
from mmaction.structures import ActionDataSample
from mmaction.testing import get_recognizer_cfg
from mmaction.utils import register_all_modules
from mmaction.utils.gradcam_utils import GradCAM
import mmaction
print(mmaction.__version__)

import mmcv
print(mmcv.__version__)

    # Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

    # Check MMEngine installation
from mmengine.utils.dl_utils import collect_env
print(collect_env())

from mmaction.apis import inference_recognizer, init_recognizer
from mmengine import Config

from operator import itemgetter

import os.path as osp
import mmengine
from mmengine.runner import Runner


1.2.0
2.2.0
12.4
MSVC 194134120
OrderedDict([('sys.platform', 'win32'), ('Python', '3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]'), ('CUDA available', True), ('MUSA available', False), ('numpy_random_seed', 2147483648), ('GPU 0', 'NVIDIA GeForce RTX 4080 SUPER'), ('CUDA_HOME', 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.4'), ('NVCC', 'Cuda compilation tools, release 12.4, V12.4.99'), ('MSVC', 'n/a, reason: fileno'), ('PyTorch', '2.4.1+cu124'), ('PyTorch compiling details', 'PyTorch built with:\n  - C++ Version: 201703\n  - MSVC 192930154\n  - Intel(R) oneAPI Math Kernel Library Version 2024.2.1-Product Build 20240722 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v3.4.2 (Git Hash 1137e04ec0b5251ca2b4400a4fd3c667ce843d67)\n  - OpenMP 2019\n  - LAPACK is enabled (usually provided by MKL)\n  - CPU capability usage: AVX2\n  - CUDA Runtime 12.4\n  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_5

In [12]:
def _get_target_shapes(input_shape, num_classes=400, model_type='2D'):
    if model_type not in ['2D', '3D']:
        raise ValueError(f'Data type {model_type} is not available')

    preds_target_shape = (input_shape[0], num_classes)
    if model_type == '3D':
        # input shape (batch_size, num_crops*num_clips, C, clip_len, H, W)
        # target shape (batch_size*num_crops*num_clips, clip_len, H, W, C)
        blended_imgs_target_shape = (input_shape[0] * input_shape[1],
                                     input_shape[3], input_shape[4],
                                     input_shape[5], input_shape[2])
    else:
        # input shape (batch_size, num_segments, C, H, W)
        # target shape (batch_size, num_segments, H, W, C)
        blended_imgs_target_shape = (input_shape[0], input_shape[1],
                                     input_shape[3], input_shape[4],
                                     input_shape[2])

    return blended_imgs_target_shape, preds_target_shape


In [13]:
def _do_test_3D_models(recognizer,
                       target_layer_name,
                       input_shape,
                       num_classes=400):
    blended_imgs_target_shape, preds_target_shape = _get_target_shapes(
        input_shape, num_classes=num_classes, model_type='3D')
    demo_data = {
        'inputs': [torch.randint(0, 256, input_shape[1:])],
        'data_samples': [ActionDataSample().set_gt_label(2)]
    }

    gradcam = GradCAM(recognizer, target_layer_name)

    blended_imgs, preds = gradcam(demo_data)
    assert blended_imgs.size() == blended_imgs_target_shape
    assert preds.size() == preds_target_shape

    blended_imgs, preds = gradcam(demo_data, True)
    assert blended_imgs.size() == blended_imgs_target_shape
    assert preds.size() == preds_target_shape



In [27]:
config = get_recognizer_cfg(
        '/Users/eddie/Trauma_THOMPSON/Trauma_THOMPSON/VideoSwin/uniformerv2_action_recognition.py')
# config.model['backbone']['pretrained2d'] = False
# config.model['backbone']['pretrained'] = None

recognizer = init_recognizer(config=config,checkpoint=None,device='cuda:0')
recognizer.cfg = config

input_shape = (1, 3, 32, 224, 224)
target_layer_name = 'backbone/transformer/norm'
print(recognizer)

_do_test_3D_models(recognizer, target_layer_name, input_shape)

11/21 19:03:17 - mmengine - INFO - Drop path rate: 0.0
11/21 19:03:17 - mmengine - INFO - No L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Double L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Drop path rate: 0.0
11/21 19:03:17 - mmengine - INFO - No L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Double L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Drop path rate: 0.0
11/21 19:03:17 - mmengine - INFO - No L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Double L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Drop path rate: 0.0
11/21 19:03:17 - mmengine - INFO - No L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Double L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Drop path rate: 0.0
11/21 19:03:17 - mmengine - INFO - No L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Double L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Drop path rate: 0.0
11/21 19:03:17 - mmengine - INFO - No L_MHRA: True
11/21 19:03:17 - mmengine - INFO - Double L_MHRA: True
11/21 19:03:17 - mmengine - INFO -

IndexError: tuple index out of range